#**Building function to build model:**


*   Resnet Block
*   Densnet Block
*  Upsampling Block
* Convolution Block
* Generator
* Discriminator



In [1]:
!pip install tensorflow-addons
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Add, Lambda, LeakyReLU, Flatten, Dense,PReLU,Concatenate
from tensorflow.keras.layers import PReLU
import tensorflow_addons as tfa
from tensorflow_addons.layers import SpectralNormalization
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt

     |████████████████████████████████| 1.1 MB 11.6 MB/s 


Convolution Block for generator include 

1.   Conv2D
2.    Activation layer (PRelu, leakyRelu)



In [ ]:
def convBlock(inputs,num_ker=64,ker_size=3,active='prelu',strides=1,name='conv'):
   x=Conv2D(filters=num_ker,kernel_size=ker_size,padding='same',strides=strides,name=name)(inputs)
   if active=='prelu':
      x=PReLU(alpha_initializer=tf.constant_initializer(0.2),shared_axes=(1,2),name='PRelu_'+name)(x)
   elif active=='leakyrelu':
      x=LeakyReLU(alpha=0.2,name='leakyRelu_'+name)(x)
   return x   
    

DenseBlock include number (no_conv_block) of ConvBlock with Concatenate layer to implement Dense Architecture

In [ ]:
#Residual in residual Dense Block
def DenseBlock(inputs,no_conv_block=5,no_filter=64,ker_size=3,name='denseBlock'):
  x=convBlock(inputs=inputs,num_ker=no_filter,ker_size=ker_size,active='prelu',name='conv.0_'+name)
  for i in range(no_conv_block-1):
    x=Concatenate(trainable=False,axis=-1)([inputs,x])
    if(i<no_conv_block-2):
      inputs=convBlock(inputs=x,num_ker=no_filter,ker_size=ker_size,active='prelu',name='conv.'+str(i+1)+'_'+name)
    else:
      inputs=convBlock(inputs=x,num_ker=no_filter,ker_size=ker_size,active='none',name='conv.'+str(i+1)+'_'+name)
  return inputs

ResnetBlock include number (no_Block) of denseBlock with Add() layer to implement Resnet Architecture, this block also called "RDBB" (Residual in residual dense block)

In [ ]:
def ResnetBlock(inputs,no_Block=3,beta=0.2,name='resnetBlock'):
  x=DenseBlock(inputs,name='denseBlock.0_'+name)
  x=Add(trainable=False)([inputs,x*beta])
  for i in range(no_Block-1):
    x=DenseBlock(x,name='denseBlock.'+str(i+1)+'_'+name)
    x=Add(trainable=False)([inputs,x*beta])
  return Add(trainable=False)([inputs,x*beta])

UpsamplingBlock is implemented of Sub-pixel convolution (Shuffle-pixel) to upsampling fearture size

In [ ]:
def upsamplingBlock(inputs,input_dimens=64,scale=2,name='shuffle'):
  x=convBlock(inputs,num_ker=input_dimens*(scale**2),ker_size=3,active='prelu',name='conv.0_'+name)
  x=tf.nn.depth_to_space(x, scale)
  x=PReLU(alpha_initializer=tf.constant_initializer(0.2),shared_axes=(1,2),name='PRelu.0_'+name)(x)
  return x

discriminatorConvBlock include convBlock for Discriminator model

In [ ]:
def discriminatorConvBlock(inputs,ker_size=3,num_ker=64,with_bn=True,name='convblock'):
  x=convBlock(inputs,ker_size=ker_size,num_ker=num_ker,active='leakyrelu',strides=1,name=name)
  if with_bn:
    x=BatchNormalization(momentum=0.8)(x)
  x=convBlock(x,ker_size=ker_size,num_ker=num_ker,active='leakyrelu',strides=2,name='strided_'+name)
  return x

In [ ]:
tf.keras.backend.clear_session()

Building Generator

In [ ]:
def build_generator():
  inputs = keras.Input(shape=(None,None, 3), name="img")
  x=x_0=convBlock(inputs,num_ker=64,ker_size=9,active='prelu',name='conv_extraction')
  x=ResnetBlock(x,name='resnetBlock.0')
  for i in range(7):
    x=ResnetBlock(x,name='resnetBlock.'+str(i+1))
  scale=2

  x=convBlock(x,num_ker=64,ker_size=3,active='none',name='conv_mapping')
  x=Add(trainable=False)([x_0,x])
  for i in range(scale):
    x= upsamplingBlock(x,scale=2,input_dimens=64,name='shuffle.'+str(i))
  x=convBlock(x,num_ker=3,ker_size=9,active='prelu',name='conv_recontructed')
  model=keras.Model(inputs,x,name='generator')
  model.summary()
  return model

Building Discriminator

In [ ]:
def build_discriminator():
  inputs=keras.Input(shape=(128,128,3),name='img')
  num_ker=64
  x=inputs
  for i in range(4):
    x=discriminatorConvBlock(x,ker_size=3,num_ker=num_ker,with_bn=False,name='conv.'+str(i))
    num_ker=num_ker*2
  x=Flatten()(x)
  x=Dense(1024)(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = Dense(1, activation='sigmoid')(x)
  discriminator=keras.Model(inputs=inputs,outputs=x,name='discriminator')
  discriminator.summary()
  return discriminator

##**Building Loss Function include:**


*   GAN Loss
*   Perceptual Loss



In [ ]:
def loss_compute(type='l1'):
    """pixel loss"""
    if type == 'l1':
        return tf.keras.losses.MeanAbsoluteError()
    else:
        return tf.keras.losses.MeanSquaredError()


In [ ]:
def featuremap_vgg19(index=5,with_activation=False):
  #vgg19_54, conv 4th before activation, after 5th maxPooling (20 th layers)
  #vgg19_22, conv 2th befor activation, after 2th maxpooling (5th layers) 
    vgg19 = tf.keras.applications.VGG19(include_top=False,input_shape=(None,None,3), weights='imagenet')
    if with_activation==False:
      vgg19.layers[index].activation=None
    perceptual_model=tf.keras.Model(inputs=vgg19.input,outputs=vgg19.layers[index].output)
    return perceptual_model

In [ ]:
def style_loss(type_loss='l2', index_layer=5, with_activation=False):
    """content loss"""
    loss_function=loss_compute(type=type_loss)
    extract_model=featuremap_vgg19(index=index_layer,with_activation=with_activation)
    @tf.function
    def content_loss(hr, sr):
        # the input scale range is [0, 1] (vgg is [0, 255]).
        # 12.75 is rescale factor for vgg featuremaps.
        preprocess_sr = tf.keras.applications.vgg19.preprocess_input(sr * 255.) 
        preprocess_hr = tf.keras.applications.vgg19.preprocess_input(hr * 255.) 
        sr_features = gram_matrix(extract_model(preprocess_sr))/ 12.75
        hr_features = gram_matrix(extract_model(preprocess_hr))/ 12.75

        return loss_function(hr_features, sr_features)

    return content_loss

In [ ]:
def feature_loss(type_loss='l2', index_layer=5, with_activation=False):
    """content loss"""
    loss_function=loss_compute(type=type_loss)
    extract_model=featuremap_vgg19(index=index_layer,with_activation=with_activation)
    @tf.function
    def content_loss(hr, sr):
        # the input scale range is [0, 1] (vgg is [0, 255]).
        # 12.75 is rescale factor for vgg featuremaps.
        preprocess_sr = tf.keras.applications.vgg19.preprocess_input(sr * 255.) 
        preprocess_hr = tf.keras.applications.vgg19.preprocess_input(hr * 255.) 
        sr_features = extract_model(preprocess_sr) / 12.75
        hr_features = extract_model(preprocess_hr) / 12.75

        return loss_function(hr_features, sr_features)

    return content_loss


#Adversarial loss

In [38]:
#type loss with = 'gan','lsgan','wgan','hingegan'
def discriminator_loss(type_loss='gan',with_relativistic=True):
  if with_relativistic==True:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        sr_logits=(sr_logits - tf.reduce_mean(hr_logits))
        hr_logits=(hr_logits - tf.reduce_mean(sr_logits))

        loss_hr=tf.reduce_mean(tf.square(hr_logits - 1.0))
        loss_sr=tf.reduce_mean(tf.square(sr_logits + 1.0))
        return (loss_hr,loss_sr)
      return lsgan_loss

    elif type_loss=='hinge':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        sr_logits=(sr_logits - tf.reduce_mean(hr_logits))
        hr_logits=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.nn.relu(1.0 - hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.relu(1.0 + sr_logits))
        return (loss_hr,loss_sr)
      return hingegan_loss

    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        sr_logits=(sr_logits - tf.reduce_mean(hr_logits))
        hr_logits=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(hr_logits), logits=hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(sr_logits), logits=sr_logits))
        return (loss_hr,loss_sr)
      return standard_loss
  ## without Relativistic Average
  else:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        loss_hr=tf.reduce_mean(tf.square(hr_logits - 1.0))
        loss_sr=tf.reduce_mean(tf.square(sr_logits ))
        return (loss_hr,loss_sr)
      return lsgan_loss

    elif type_loss=='hinge':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        loss_hr=tf.reduce_mean(tf.nn.relu(1.0 - hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.relu(1.0 + sr_logits ))
        return (loss_hr,loss_sr)
      return hingegan_loss

    elif type_loss=='wgan':
      @tf.function
      def wgan_loss(sr_logits,hr_logits):
        loss_hr=-tf.reduce_mean(hr_logits)
        loss_sr=tf.reduce_mean(sr_logits)
        return (loss_hr,loss_sr)
      return wgan_loss
    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        loss_hr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(hr_logits), logits=hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(sr_logits), logits=sr_logits))
        return (loss_hr,loss_sr)
      return standard_loss

In [41]:
#type loss with = 'gan','lsgan','wgan','hingegan'
def generator_loss(type_loss='gan',with_relativistic=True):
  if with_relativistic==True:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        sr_logits=(sr_logits - tf.reduce_mean(hr_logits))
        hr_logits=(hr_logits - tf.reduce_mean(sr_logits))

        loss_hr=tf.reduce_mean(tf.square(hr_logits + 1.0))
        # hr --> -1.0
        loss_sr=tf.reduce_mean(tf.square(sr_logits - 1.0))
        # sr --> 1.0
        return (loss_hr,loss_sr)
      return lsgan_loss

    elif type_loss=='hinge':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        sr_logits=(sr_logits - tf.reduce_mean(hr_logits))
        hr_logits=(hr_logits - tf.reduce_mean(sr_logits))

        loss_hr=tf.reduce_mean(tf.nn.relu(1.0 + hr_logits))
        # hr --> -1.0
        loss_sr=tf.reduce_mean(tf.nn.relu(1.0 - sr_logits))
        # sr --> 1.0
        return (loss_hr,loss_sr)
      return hingegan_loss

    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        sr_logits=(sr_logits - tf.reduce_mean(hr_logits))
        hr_logits=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(hr_logits), logits=hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(sr_logits), logits=sr_logits))
        return (loss_hr,loss_sr)
      return standard_loss
  ## without Relativistic Average
  else:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        return tf.reduce_mean(tf.square(sr_logits - 1.0))
      return lsgan_loss
    elif type_loss=='hinge' or type_loss=='wgan':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        return -tf.reduce_mean(sr_logits)
      return hingegan_loss
    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(sr_logits), logits=sr_logits))
      return standard_loss

In [45]:
loss=generator_loss(type_loss='lsgan',with_relativistic=False)
sr= tf.constant([1.8, 2.2], dtype=tf.float32)
hr =tf.constant([1.8, 2.2], dtype=tf.float32)
loss(sr,hr)

<tf.Tensor: shape=(), dtype=float32, numpy=1.04>

#Operator

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

In [ ]:
def imshow(image, title=None):
  plt.figure(figsize=(10, 10))
  plt.imshow(image[0,:,:,:])

In [ ]:
def show_feature(input,number=8,title=None):
  # plot all 64 maps in an 8x8 squares
  square = number
  ix = 1
  plt.figure(figsize=(4*number, 4*number))
  for _ in range(square):
    for _ in range(square):
		# specify subplot and turn of axis
      ax = plt.subplot(square, square, ix)
      ax.set(title=ix)
      plt.imshow(input[0, :, :, ix-1], cmap='gray',aspect='equal')
      ix += 1
  plt.show()
# show the figure

#Metrics